-오산시 좌표계 : 37.149799, 127.077116  
-'Stamen Toner', 'Cartodb Positron'  
-PuRd, YlGnBu


In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import math
import matplotlib.pyplot as plt

%matplotlib inline

import folium
import geopandas as gpd

import warnings
warnings.filterwarnings(action='ignore')

In [2]:
data1 = pd.read_csv('1.오산시_주정차단속(2018~2020).csv')
data15 = pd.read_csv('15.오산시_무인교통단속카메라.csv')
data20 = pd.read_csv('20.오산시_CCTV설치현황.csv')

### (1)오산시_주정차단속(2018~2020) ###
+ 주정차가 많으면 시야 확보가 어려워져 교통사고의 위험이 증가함.

In [3]:
data1.tail()

,단속일자,단속위치,행정구역,스쿨존여부,단속위치_경도,단속위치_위도,단속종류
61696,20201111,GS25궐동대호점 맞은편,경기도 오산시 청학동,N,127.053064,37.158140,고정형CCTV단속
61697,20201111,물향기지하도,경기도 오산시 원동,N,127.066157,37.171143,고정형CCTV단속
61698,20201111,운암2단지 근린공원삼거,경기도 오산시 청호동,N,127.077283,37.158126,고정형CCTV단속
61699,20201111,궐동 양평해장국 사거리,경기도 오산시 궐동,N,127.056665,37.155827,고정형CCTV단속
61700,20201111,어울림(북개천) 사거리,경기도 오산시 부산동,N,127.068644,37.143810,고정형CCTV단속


In [4]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61701 entries, 0 to 61700
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   단속일자     61701 non-null  int64  
 1   단속위치     61701 non-null  object 
 2   행정구역     61701 non-null  object 
 3   스쿨존여부    61701 non-null  object 
 4   단속위치_경도  61513 non-null  float64
 5   단속위치_위도  61513 non-null  float64
 6   단속종류     61701 non-null  object 
dtypes: float64(2), int64(1), object(4)
memory usage: 3.3+ MB


In [5]:
data1.describe()

,단속일자,단속위치_경도,단속위치_위도
count,6.170100e+04,61513.000000,61513.000000
mean,2.019323e+07,127.061603,37.158325
std,6.126415e+03,0.011035,0.013492
min,2.018110e+07,127.023123,37.127578
25%,2.019041e+07,127.053626,37.148675
50%,2.019092e+07,127.063520,37.158101
75%,2.020032e+07,127.070185,37.169386
max,2.020111e+07,127.095988,37.199590


In [6]:
print('<단속위치> \n', data1['단속위치'].value_counts())
print('<행정구역> \n', data1['행정구역'].value_counts()) #오산시 아닌 구역 존재
print('<스쿨존여부> \n', data1['스쿨존여부'].value_counts())
print('<단속종류> \n', data1['단속종류'].value_counts()) # 고정형 CCTV가 설치된 구역에서 많이 발생

<단속위치> 
 GS25궐동대호점 맞은편    2824
GS25궐동승리점 앞      2354
GS25오산타운점 맞은편    2140
세교 본초한의원 앞       2093
유시티상황실           1764
                 ... 
내삼미동 산 69 부근        1
양산동 154 부근          1
수청동 오산로 부근          1
수청동 107 부근          1
세교동 세마역로19번길        1
Name: 단속위치, Length: 351, dtype: int64
<행정구역> 
 경기도 오산시 궐동              12494
경기도 오산시 원동               8597
경기도 오산시 금암동              5936
경기도 오산시 오산동              5289
경기도 오산시 수청동              4289
경기도 오산시 내삼미동             3979
경기도 오산시 부산동              2971
경기도 오산시 갈곶동              2660
경기도 오산시 세교동              1918
경기도 오산시 지곶동              1707
경기도 오산시 누읍동              1596
경기도 오산시 은계동              1582
경기도 오산시 청호동              1442
경기도 오산시 가수동              1230
경기도 오산시 양산동              1217
경기도 오산시 서동               1192
경기도 오산시 청학동               901
경기도 오산시 고현동               759
경기도 화성시 오산동               636
경기도 오산시 외삼미동              632
경기도 오산시 가장동               261
경기도 오산시 두곡동               106
경기도 오산시 벌음동                94
경기도 오산시 탑동  

In [7]:
# 오산시가 아닌 구역 제거
droprow = []
for idx in data1.index:
    if data1['행정구역'][idx].find('오산시') == -1 :
        droprow.append(idx)

droprow

[117,
 161,
 203,
 298,
 486,
 657,
 727,
 827,
 864,
 1260,
 1357,
 1722,
 1738,
 1740,
 1961,
 2089,
 2213,
 2562,
 2872,
 2973,
 3128,
 3152,
 3166,
 3220,
 3237,
 3238,
 3243,
 3276,
 3291,
 3336,
 3337,
 3352,
 3414,
 3479,
 3736,
 3897,
 4067,
 4094,
 4187,
 4347,
 4388,
 4522,
 4615,
 4678,
 4834,
 4847,
 4897,
 4913,
 4924,
 4955,
 5085,
 5153,
 5258,
 5275,
 5735,
 5763,
 5819,
 5938,
 5960,
 5995,
 5999,
 6058,
 6152,
 6179,
 6334,
 6474,
 6498,
 6728,
 6788,
 6871,
 6936,
 6990,
 7213,
 7281,
 7347,
 7406,
 7450,
 7501,
 7575,
 7748,
 7802,
 7893,
 8007,
 8069,
 8105,
 8208,
 8329,
 8359,
 8513,
 8514,
 8560,
 8666,
 8838,
 8846,
 8848,
 8878,
 8982,
 9013,
 9087,
 9144,
 9231,
 9237,
 9253,
 9351,
 9386,
 9467,
 9564,
 9634,
 9638,
 9667,
 9833,
 9841,
 9946,
 10044,
 10073,
 10184,
 10224,
 10295,
 10324,
 10330,
 10489,
 10497,
 10503,
 10504,
 10525,
 10577,
 10700,
 10771,
 10874,
 10961,
 11005,
 11117,
 11261,
 11263,
 11268,
 11272,
 11375,
 11384,
 11394,
 11478,
 1

In [8]:
data1_before = data1.copy()

data1 = data1.drop(droprow)
data1['행정구역'].value_counts()

경기도 오산시 궐동      12494
경기도 오산시 원동       8597
경기도 오산시 금암동      5936
경기도 오산시 오산동      5289
경기도 오산시 수청동      4289
경기도 오산시 내삼미동     3979
경기도 오산시 부산동      2971
경기도 오산시 갈곶동      2660
경기도 오산시 세교동      1918
경기도 오산시 지곶동      1707
경기도 오산시 누읍동      1596
경기도 오산시 은계동      1582
경기도 오산시 청호동      1442
경기도 오산시 가수동      1230
경기도 오산시 양산동      1217
경기도 오산시 서동       1192
경기도 오산시 청학동       901
경기도 오산시 고현동       759
경기도 오산시 외삼미동      632
경기도 오산시 가장동       261
경기도 오산시 두곡동       106
경기도 오산시 벌음동        94
경기도 오산시 탑동         93
경기도 오산시 서랑동        88
Name: 행정구역, dtype: int64

In [9]:
# 주정차단속수 분포 확인 -> 지도에 표시
data1[['단속위치_위도','단속위치_경도']].isnull().sum()

단속위치_위도    187
단속위치_경도    187
dtype: int64

In [10]:
m = folium.Map(location = [37.149799, 127.077116], zoom_start = 12)

import math

for idx, row in data1.iterrows():
    if math.isnan(row['단속위치_위도']) == False:
        folium.Circle(location = [row['단속위치_위도'],row['단속위치_경도']], radius = 10, color = 'blue',
                      fill = 'crimson', popup = row['행정구역']).add_to(m)
m.save('주정차단속수 분포.html')

In [11]:
# 포인트보다는 구역별로 보는게 한눈에 잘 보일듯

In [12]:
data32 = gpd.read_file('32.오산시_행정경계(읍면동).geojson')
data32['ADM_DR_NM'].value_counts()

대원동    1
세마동    1
중앙동    1
신장동    1
초평동    1
남촌동    1
Name: ADM_DR_NM, dtype: int64

In [13]:
data33 = gpd.read_file('33.오산시_법정경계(읍면동).geojson')
data33['EMD_KOR_NM'].value_counts()

서동      1
궐동      1
가수동     1
지곶동     1
가장동     1
고현동     1
세교동     1
내삼미동    1
부산동     1
양산동     1
벌음동     1
원동      1
누읍동     1
청학동     1
갈곶동     1
탑동      1
금암동     1
오산동     1
청호동     1
은계동     1
서랑동     1
외삼미동    1
수청동     1
두곡동     1
Name: EMD_KOR_NM, dtype: int64

In [14]:
# 법정경계별 주정차단속수 비교 
# data1['행정구역'] : '경기도 오산시 OO동'
# data33['EMD_KOR_NM'] : 'OO동'
data1['area'] = ''
for idx, row in data1.iterrows():
    string = row['행정구역'].split(' ')[2]
    data1['area'][idx] = string
data1.tail()

,단속일자,단속위치,행정구역,스쿨존여부,단속위치_경도,단속위치_위도,단속종류,area
61696,20201111,GS25궐동대호점 맞은편,경기도 오산시 청학동,N,127.053064,37.158140,고정형CCTV단속,청학동
61697,20201111,물향기지하도,경기도 오산시 원동,N,127.066157,37.171143,고정형CCTV단속,원동
61698,20201111,운암2단지 근린공원삼거,경기도 오산시 청호동,N,127.077283,37.158126,고정형CCTV단속,청호동
61699,20201111,궐동 양평해장국 사거리,경기도 오산시 궐동,N,127.056665,37.155827,고정형CCTV단속,궐동
61700,20201111,어울림(북개천) 사거리,경기도 오산시 부산동,N,127.068644,37.143810,고정형CCTV단속,부산동


In [15]:
data1['area'].isin(data33['EMD_KOR_NM']).value_counts()

True    61033
Name: area, dtype: int64

In [16]:
# col별 count 해주는 함수
def count_group(df, col):
    count_df = pd.DataFrame(df[col].groupby(df[col]).count())
    count_df.columns = ['count']
    count_df.reset_index(inplace = True)
    return count_df

In [17]:
area_df = count_group(data1, 'area')

In [18]:
area_df.sort_values(by = 'count', ascending = False).head(5)

,area,count
4,궐동,12494
18,원동,8597
5,금암동,5936
16,오산동,5289
14,수청동,4289


In [19]:
m = folium.Map(location = [37.149799, 127.077116], zoom_start = 13, tiles = 'Cartodb Positron')

folium.Choropleth(geo_data = data33, data = area_df, columns = ('area', 'count'), key_on='feature.properties.EMD_KOR_NM',
    fill_color = 'YlGnBu', legend_name='주정차단속수').add_to(m)

m

- 단속일자

In [20]:
data1['단속일자'] = data1['단속일자'].astype('str')
data1['단속일자'] = pd.to_datetime(data1['단속일자'])

data1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61033 entries, 0 to 61700
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   단속일자     61033 non-null  datetime64[ns]
 1   단속위치     61033 non-null  object        
 2   행정구역     61033 non-null  object        
 3   스쿨존여부    61033 non-null  object        
 4   단속위치_경도  60846 non-null  float64       
 5   단속위치_위도  60846 non-null  float64       
 6   단속종류     61033 non-null  object        
 7   area     61033 non-null  object        
dtypes: datetime64[ns](1), float64(2), object(5)
memory usage: 6.7+ MB


In [21]:
data1['단속일자']

0       2018-11-01
1       2018-11-01
2       2018-11-01
3       2018-11-01
4       2018-11-01
           ...    
61696   2020-11-11
61697   2020-11-11
61698   2020-11-11
61699   2020-11-11
61700   2020-11-11
Name: 단속일자, Length: 61033, dtype: datetime64[ns]

In [22]:
date_df = pd.DataFrame(data1['단속일자'].groupby(data1['단속일자']).count())

date_df.columns = ['count']
date_df.reset_index(inplace = True)

date_df.sort_values(by = 'count', ascending = False)

,단속일자,count
162,2019-04-17,158
211,2019-06-05,156
176,2019-05-01,153
219,2019-06-13,150
344,2019-10-21,149
...,...,...
505,2020-04-04,6
512,2020-04-11,5
495,2020-03-25,5
540,2020-05-09,5


In [23]:
# 요일 추출별 주정차단속수
import datetime

data1['Weekday'] = ''
daylist = ['월','화','수','목','금','토','일']

for idx in data1.index:
    data1['Weekday'][idx] = daylist[data1['단속일자'][idx].weekday()]

data1['Weekday'].value_counts()

# 평일 > 주말

화    10317
월    10129
수     9581
금     9343
목     9209
일     6743
토     5711
Name: Weekday, dtype: int64

- 스쿨존 여부

In [24]:
data1['스쿨존여부'].value_counts()

N    60956
Y       77
Name: 스쿨존여부, dtype: int64

In [25]:
data1_N = data1[data1['스쿨존여부'] == 'N']

N_count = count_group(data1_N, 'area')
N_count.sort_values(by = 'count', ascending = False).head(5)

,area,count
4,궐동,12487
18,원동,8591
5,금암동,5935
16,오산동,5288
14,수청동,4288


In [26]:
m = folium.Map(location = [37.149799, 127.077116], zoom_start = 13)#, tiles = 'Stamen Toner')

folium.Choropleth(geo_data = data33, data = N_count, columns = ('area', 'count'), key_on='feature.properties.EMD_KOR_NM',
    fill_color = 'YlGnBu', legend_name='스쿨존이 아닌 구역에서의 주정차단속수').add_to(m)

m
# 스쿨존여부로 나누지 않았을 때와 결과 동일

In [27]:
# 스쿨존인 곳에서의 주정차위반
data1_Y = data1[data1['스쿨존여부'] == 'Y']

m = folium.Map(location = [37.149799, 127.077116], zoom_start = 13)

for idx, row in data1_Y.iterrows():
    if math.isnan(row['단속위치_위도']) == False:
        folium.Circle(location = [row['단속위치_위도'],row['단속위치_경도']], radius = 10, color = 'blue',
                      fill = 'crimson', popup = row['행정구역']).add_to(m)
m
# 가수초 & 오산초 부근
# (참고)CCTV 수 & 교통사고수.html

In [28]:
m = folium.Map(location = [37.149799, 127.077116], zoom_start = 12)

for idx, row in data1_Y.iterrows():
    if math.isnan(row['단속위치_위도']) == False:
        folium.Circle(location = [row['단속위치_위도'],row['단속위치_경도']], radius = 10, color = 'blue',
                      fill = 'crimson', popup = row['행정구역']).add_to(m)

for idx in data15.index:
    folium.Marker([data15['설치위치_위도'][idx],data15['설치위치_경도'][idx]], 
                  icon = folium.Icon(icon = 'camera',color = 'red')).add_to(m)

m

In [29]:
data1_Y['단속종류'].value_counts()
# 가수초 & 오산초 부근 무인교통단속카메라(고정형CCTV단속) 추가 설치하는 것이 좋을 듯?

이동형CCTV단속    77
Name: 단속종류, dtype: int64

### (15) 15.오산시_무인교통단속카메라 ###

In [30]:
data15.tail()

,도로노선명,도로노선방향,설치장소명,단속구분,설치위치_경도,설치위치_위도
107,경기동로186번길,양방향,시티자이2차 아파트 앞,불법주정차,127.091941,37.154632
108,역광장로,양방향,오원사거리,불법주정차,127.068303,37.142519
109,청학로,양방향,차량등록사업소,불법주정차,127.064287,37.158197
110,수목원로468번길,양방향,세교9단지 후문 상가도로,불법주정차,127.050714,37.175949
111,원동로,양방향,운암고등학교,불법주정차,127.074068,37.143045


In [31]:
data15['단속구분'].value_counts() # 불법주정차 단속 CCTV에 대한 파일임을 확인

불법주정차    112
Name: 단속구분, dtype: int64

In [32]:
print(data15.info())
print(data15.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112 entries, 0 to 111
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   도로노선명    112 non-null    object 
 1   도로노선방향   112 non-null    object 
 2   설치장소명    112 non-null    object 
 3   단속구분     112 non-null    object 
 4   설치위치_경도  112 non-null    float64
 5   설치위치_위도  112 non-null    float64
dtypes: float64(2), object(4)
memory usage: 5.4+ KB
None
          설치위치_경도     설치위치_위도
count  112.000000  112.000000
mean   127.064562   37.158971
std      0.013930    0.015221
min    127.022971   37.129890
25%    127.055731   37.148268
50%    127.066490   37.158035
75%    127.073856   37.169724
max    127.091941   37.196268


In [33]:
# 무인교통단속카메라 위치
m = folium.Map(location = [37.149799, 127.077116], zoom_start = 13)

for idx in data15.index:
    folium.Circle(location = [data15['설치위치_위도'][idx],data15['설치위치_경도'][idx]], radius = 15, color = 'blue',
                      fill = 'crimson', popup = data15['설치장소명'][idx]).add_to(m)

m

In [34]:
# 주정차단속수와 단속카메라 관계
m = folium.Map(location = [37.149799, 127.077116], zoom_start = 13, tiles = 'Cartodb Positron')

folium.Choropleth(geo_data = data33, data = area_df, columns = ('area', 'count'), key_on='feature.properties.EMD_KOR_NM',
    fill_color = 'YlGnBu', legend_name='주정차단속수').add_to(m)

for idx in data15.index:
    folium.Circle(location = [data15['설치위치_위도'][idx],data15['설치위치_경도'][idx]], radius = 15, color = 'red',
                      fill = 'crimson', popup = data15['설치장소명'][idx]).add_to(m)

m

# 주정차단속수가 적은 위치는 CCTV의 수도 적어서가 아닐까? 
# -> 만약 CCTV수가 적은 구역에서 교통사고가 꽤 일어난다면 CCTV설치를 늘려야 할것

### (20) 20.오산시_CCTV설치현황 ###
- A : 방범, B : 도시공원, C : 어린이보호, D : 차량방범, E : 과속단속

In [35]:
data20
# 위도, 경도 값 확인 필요

,CCTV 유형코드,설치위치_도로명주소,설치위치_위도,설치위치_경도
0,A,성호대로55번길 31-2,37.151836,127.067541
1,A,성호대로93번길 45,37.152812,127.071702
2,C,성호대로93번길 26,37.151167,127.071677
3,A,운천로165번길 52-1,37.157164,127.075991
4,A,운천로165번길 5,37.156461,127.077273
...,...,...,...,...
562,E,성호초,127.070001,37.149096
563,E,대원초,127.072728,37.131781
564,E,원일초,127.071237,37.139445
565,E,운천초,127.078048,37.156239


In [36]:
data20['CCTV 유형코드'].value_counts()

A     330
C     125
B      98
E      10
D       2
BC      1
 B      1
Name: CCTV 유형코드, dtype: int64

In [37]:
data20.replace(' B','B',inplace = True)
data20.drop(data20.loc[data20['CCTV 유형코드'] == 'BC'].index, inplace = True)

In [38]:
data20['CCTV 유형코드'].value_counts()

A    330
C    125
B     99
E     10
D      2
Name: CCTV 유형코드, dtype: int64

In [39]:
data20_C = data20[data20['CCTV 유형코드'] == 'C']
data20_E = data20[data20['CCTV 유형코드'] == 'E']

In [40]:
# C 위도 경도 값 확인, 정리
print(data20_C.info())
print(data20_C.describe())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 125 entries, 2 to 464
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   CCTV 유형코드   125 non-null    object 
 1   설치위치_도로명주소  91 non-null     object 
 2   설치위치_위도     125 non-null    float64
 3   설치위치_경도     125 non-null    float64
dtypes: float64(2), object(2)
memory usage: 4.9+ KB
None
          설치위치_위도     설치위치_경도
count  125.000000  125.000000
mean    37.157015  127.422658
std      0.017834    4.024612
min     37.127435  127.016879
25%     37.142729  127.053631
50%     37.155427  127.065719
75%     37.170053  127.072348
max     37.198166  172.058905


In [41]:
data20_C.replace(172.058905, 127.058905, inplace = True)
data20_C.describe()

,설치위치_위도,설치위치_경도
count,125.000000,125.000000
mean,37.157015,127.062658
std,0.017834,0.014945
min,37.127435,127.016879
25%,37.142729,127.053631
50%,37.155427,127.065449
75%,37.170053,127.072204
max,37.198166,127.091524


In [42]:
# E 위도 경도 값 확인, 정리
print(data20_E.info())
print(data20_E.describe())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 557 to 566
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   CCTV 유형코드   10 non-null     object 
 1   설치위치_도로명주소  10 non-null     object 
 2   설치위치_위도     10 non-null     float64
 3   설치위치_경도     10 non-null     float64
dtypes: float64(2), object(2)
memory usage: 400.0+ bytes
None
          설치위치_위도    설치위치_경도
count   10.000000  10.000000
mean   127.075719  37.148845
std      0.010583   0.008018
min    127.057906  37.131781
25%    127.070310  37.148526
50%    127.074469  37.150190
75%    127.082510  37.151984
max    127.090748  37.159936


In [43]:
data20_E.rename(columns = {'설치위치_위도':'설치위치_경도', '설치위치_경도':'설치위치_위도'}, inplace = True)

In [44]:
m = folium.Map(location = [37.149799, 127.077116], zoom_start = 13)

for idx in data20_C.index:
    folium.Marker([data20_C['설치위치_위도'][idx],data20_C['설치위치_경도'][idx]], 
                  popup = data20_C['설치위치_도로명주소'][idx], icon = folium.Icon(icon = 'camera',color = 'blue')).add_to(m)

for idx in data20_E.index:
    folium.Marker([data20_E['설치위치_위도'][idx],data20_E['설치위치_경도'][idx]], 
                  popup = data20_E['설치위치_도로명주소'][idx], icon = folium.Icon(icon = 'camera',color = 'red')).add_to(m)

m

In [54]:
data9 = pd.read_csv('9.오산시_어린이보호구역.csv')
data9

,시설종류,시설명,CCTV설치여부,CCTV설치대수,보호구역도로폭,보호구역_경도,보호구역_위도
0,특수학교,성심학교,N,0,25,127.017521,37.176762
1,초등학교,가수초등학교,Y,1,12,127.047957,37.148103
2,초등학교,광성초등학교,Y,2,12,127.039709,37.184197
3,초등학교,대호초등학교,Y,3,12,127.057889,37.159106
4,초등학교,매홀초등학교,Y,2,6,127.063282,37.162749
...,...,...,...,...,...,...,...
86,어린이집,시립세마어린이집,Y,1,12,127.044916,37.184608
87,어린이집,미래숲어린이집,Y,1,6,127.044384,37.145326
88,어린이집,시립은여울어린이집,N,0,20,127.066860,37.170952
89,어린이집,설리반어린이집,N,0,5,127.061041,37.164127


In [56]:
m = folium.Map(location = [37.149799, 127.077116], zoom_start = 13)

for idx in data9.index:
    folium.Circle(location = [data9['보호구역_위도'][idx],data9['보호구역_경도'][idx]], radius = 15, color = 'blue',
                      fill = 'crimson').add_to(m)

m